# Neural Network model for fluid dynamics (Part 2)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/paolodeangelis/Sistemi_a_combustione/blob/main/2.2-NN_Reynolds_P2.ipynb)

## Hypothetical Experiment

Let's imagine an experimental setup as depicted in the sketch below. In this hypothetical scenario, we are examining the flow of incompressible water within a duct, which exhibits various flow regimes. To gather data, we employ a fictional Pitot tube grid to obtain velocity profiles.


<img style="display: block; margin: auto;" alt="Experimental Setup" src="https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/assets/img/NN_problem.png">

The central question we pose in this hypothetical experiment is: Can we effectively utilize a neural network (NN) to analyze and interpret these hypothetical velocity profiles and estimate the Reynolds number $\mathrm{Re}$

## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to `Edit` > `Notebook Settings`
- Select `T4 GPU` from the Hardware Accelerator drop-down
- If the GPU nodes are busy, use the CPU one.

Next, we'll check that we can connect to the GPU:

In [ ]:
import warnings

import tensorflow as tf

print("Tensorflow version " + tf.__version__)

device_name = tf.test.gpu_device_name()  # GPU detection
if device_name == "/device:GPU:0":
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print(f"Running on GPU at: {device_name}")
else:
    warnings.warn("GPU device not found")
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
            tpu=""
        )  # TPU detection
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.TPUStrategy(resolver)
        print(
            "Running on TPU at:",
            "\n\t".join([f"{i}" for i in tf.config.list_logical_devices("TPU")]),
        )
    except ValueError:
        warnings.warn("TPU device not found")
        warnings.warn("Default parallization strategy will be used")
        strategy = tf.distribute.get_strategy()

## Installing Libraries

We begin by installing the necessary libraries to support our data manipulation, visualization, and deep learning modeling. (Note: `Tensorflow` and `Keras` are already installed on Colab)

In [ ]:
%pip install numpy pandas scipy matplotlib  scikit-learn

And now we import the necessary libraries

In [ ]:
import os  # Operating system-related functions
import pathlib  # Path manipulation and filesystem-related operations

import matplotlib.pyplot as plt  # Data visualization library
import numpy as np  # Numerical computing library
import pandas as pd  # Data manipulation and analysis library
import tensorflow as tf  # Deep learning framework for neural networks
from sklearn.metrics import r2_score
from tensorflow.keras import Sequential, layers, losses, optimizers

## Data download and downsamplig

In this section, we will download the dataset and  reduce its size to speed up the training.

### Download dataset files

In [ ]:
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-Re.csv
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-data.csv
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-space.csv

### Read Reynolds (the labels for our model)

We'll start by reading the Reynolds data, which serves as the labels for our model.

In [ ]:
data_Re = pd.read_csv("velprof-Re.csv", index_col=False)
data_Re.head()

### Read the data file (the features for our model)

Next, we'll read the data file containing the features for our model.

In [ ]:
data_v = pd.read_csv("velprof-data.csv", index_col=False)
data_v.head()

Let's also read another data file that contains information about space discretization, which, in our analogy, represents the probe's position.

In [ ]:
data_r = pd.read_csv("velprof-space.csv", index_col=False)
data_r.head()

We can also merge the two datasets, which include both labels and features (it can be useful later).

In [ ]:
data_all = pd.concat([data_v, data_Re], axis=1)

### Downsampling the database

The dataset contains too many data points (100,000), so we will reduce it to 40,000 by randomly selecting from the entire database.

In [ ]:
# Get the total number of data points in data_v
Nall = data_v.shape[0]

# Define the desired number of smaller data points to select
Nsmall = 40000

# Initialize a random number generator with a specific seed for reproducibility
rand_gen = np.random.default_rng(seed=1234)

# Generate a random sample of indices without replacement from the range [0, Nall)
# This will be used to select a subset of data_v and data_Re
indx = rand_gen.choice(np.arange(Nall), size=Nsmall, replace=False)

# Create smaller subsets of data_v and data_Re based on the randomly selected indices
data_v_small = data_v.iloc[indx, :]
data_Re_small = data_Re.iloc[indx, :]

let's store it as `.csv` (comma-separated values)

In [ ]:
data_v_small.to_csv("small-data.csv", index=False)
data_Re_small.to_csv("small-Re.csv", index=False)

## Model 2: Re number *regression*


In this section, we will employ our Neural-Network model for *regression* with the primary goal of predicting the reynolds number from the velocity profile.

### Setup database

We load the features and the labels of our first model.

In [ ]:
# Note: this time we include the intial coloms contaiong phisical property of the fluid, and channel sizes (mu(Pas)	rho(kg/m3)	L(m)	R(m))
features = pd.read_csv("small-data.csv", index_col=False)
labels = pd.read_csv("small-Re.csv", index_col=False)

Display the first few rows of the features.

In [ ]:
features.head()

Display the first few rows of the labels.

In [ ]:
labels.head()

To create a training and test dataset, we perform an 80/20 split.

In [ ]:
# Splitting the dataset into training and test sets
# Training set (80% of the data)
labels_train = labels.iloc[int(Nsmall * 0.2) :, :]
features_train = features.iloc[int(Nsmall * 0.2) :, :]

# Test set (20% of the data)
labels_test = labels.iloc[: int(Nsmall * 0.2), :]
features_test = features.iloc[: int(Nsmall * 0.2), :]

### Storing

We will store the datasets and labels in a structured folder for future use.

In [ ]:
pathlib.Path("model_2").mkdir(parents=True, exist_ok=True)  # Make a folder
# Make train and test subfolders
pathlib.Path(os.path.join("model_2", "train")).mkdir(exist_ok=True)
pathlib.Path(os.path.join("model_2", "test")).mkdir(exist_ok=True)
# Note: This time, we include the initial columns containing physical properties of the fluid and channel sizes (mu (Pas), rho (kg/m³), L (m), R (m)).
labels_train.to_csv(os.path.join("model_2", "train", "labels.csv"), index=False)
labels_test.to_csv(os.path.join("model_2", "test", "labels.csv"), index=False)
features_train.to_csv(os.path.join("model_2", "train", "features.csv"), index=False)
features_test.to_csv(os.path.join("model_2", "test", "features.csv"), index=False)

### First Try

The sketch below represents the proposed neural network architecture.

<img style="display: block; margin: auto;" alt="NN sketch" src="https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/assets/img/NN_model2_try1.png">

#### Architecture

#### Building the Model with Keras and TensorFlow


Now we define a function to create our neural network model using Keras and TensorFlow. This involves designing the layers, specifying their connections, and defining their behavior, including activation functions, regularization, dropout, and more.

In a neural network, a *layer* serves as a fundamental building block responsible for processing data and extracting features. Layers are combined to form the architecture of the neural network. Each layer consists of one or more "neurons" (also known as "nodes" or "units").

For our first model, we need to include the following types of layers:

* **Input Layer**: This is the initial layer that receives raw input data. Its primary role is to pass the data to subsequent layers. The input layer typically has one neuron for each feature present in the input data.

* **Hidden Layers**: These intermediate layers sit between the input and output layers. Hidden layers perform complex transformations on the data, enabling the network to learn and extract features from the input. Each neuron in a hidden layer receives input from multiple neurons in the previous layer.

* **Output Layer**: The final layer in the neural network is responsible for producing the model's predictions. The number of neurons in the output layer depends on the nature of the problem being addressed. For the Reynolds number regression ($\mathrm{Re} \in \left[0, +\infty\right)$), a single neuron is common in the output layer. Here is crucial the choice of the activation function

In addition to layers, we also introduce the concept of an *activation function*. An activation function is a critical element within each neuron of a neural network. It dictates how the output of a neuron is calculated based on its input. The activation function introduces non-linearity into the model, allowing it to learn complex patterns and relationships within the data.

In the following example we are going to use

* **ReLU (Rectified Linear Unit)**: ReLU is one of the most widely used activation functions. It returns the input value if it's positive and zero if it's negative. Mathematically, it can be represented as $f(x) = \max(0, x)$. ReLU is effective in training deep networks and addressing the vanishing gradient problem.

* **Linear**: Basicaly the output neurons gather features from the previous layer and return the result through a matrix operation $\mathbf{W}\mathbf{x} + b$ with $\mathbf{W} \in \mathbb{R}^{1\times N}$, $\mathbf{x} \in \mathbb{R}^{N\times 1}$. Mathematically, it's expressed as $f(x) = x$.


#### Load data

In [ ]:
features_train_try1 = pd.read_csv(
    os.path.join("model_2", "train", "features.csv"), index_col=False
).iloc[
    :, 4:
]  # Note: we drop the first 4 columns to study only the velocity profile
features_test_try1 = pd.read_csv(
    os.path.join("model_2", "test", "features.csv"), index_col=False
).iloc[
    :, 4:
]  # Note: we drop the first 4 columns to study only the velocity profile
labels_train_try1 = pd.read_csv(
    os.path.join("model_2", "train", "labels.csv"), index_col=False
)
labels_test_try1 = pd.read_csv(
    os.path.join("model_2", "test", "labels.csv"), index_col=False
)

In [ ]:
def build_model_2(n_cols: int, out_activation: str) -> tf.keras.models.Sequential:
    """
    Build a the first possible architecture for our neural network model.

    Args:
        n_cols (int): Number of input features.
        out_activation (str): Output activation function.
    Returns:
        tf.keras.models.Sequential: A Keras Sequential model.
    """
    model = Sequential(
        [
            # Input layer with the specified input shape
            layers.InputLayer(input_shape=(n_cols,), name="input_layer"),
            # Add the first hidden layer with 64 perceptron and ReLU activation
            layers.Dense(64, activation="relu", name="hidden_layer_1"),
            # Add the second hidden layer with 64 perceptron and ReLU activation
            layers.Dense(64, activation="relu", name="hidden_layer_2"),
            # Add the third hidden layer with 64 perceptron and ReLU activation
            layers.Dense(32, activation="relu", name="hidden_layer_3"),
            # Add the forth hidden layer with 64 perceptron and ReLU activation
            layers.Dense(32, activation="relu", name="hidden_layer_4"),
            # Add the output layer with a single perceptron (we expect a True/False answer) and sigmoid activation
            layers.Dense(1, activation=out_activation, name="output_layer"),
        ]
    )
    return model

Let's call the function to build our model.

In [ ]:
model = build_model_2(features_train_try1.shape[1], "linear")

#### Model Summary and Structure Visualization

Let's examine the summary of our compiled model and visualize its architecture.

Given that the input data has 50 features and the first hidden layer consists of 64 neurons (each with weights $w$ and biases $b$), we have the following:

- Number of parameters in the first hidden layer ($\mathbf{w}_{h1}$): $50 \times 64 = 3200$
- Number of parameters in the first hidden layer biases ($\mathbf{b}_{h1}$): $1 \times 64 = 64$

The second hidden layer is densely connected with the first one, therefore we have 64 "inputs feature" from the first hidden layer:

- Number of parameters in the first hidden layer ($\mathbf{w}_{h1}$): $64 \times 64 = 4096$
- Number of parameters in the first hidden layer biases ($\mathbf{b}_{h1}$): $1 \times 64 = 64$

The third hidden layer is densely connected with the second one, therefore we have 32 "inputs feature" from the first hidden layer:

- Number of parameters in the first hidden layer ($\mathbf{w}_{h1}$): $64 \times 32 = 2048$
- Number of parameters in the first hidden layer biases ($\mathbf{b}_{h1}$): $1 \times 32 = 32$

The forth hidden layer is densely connected with the third one, therefore we have 32 "inputs feature" from the first hidden layer:

- Number of parameters in the first hidden layer ($\mathbf{w}_{h1}$): $32 \times 32 = 1024$
- Number of parameters in the first hidden layer biases ($\mathbf{b}_{h1}$): $1 \times 32 = 32$

In addition, considering the output layer:

- Number of parameters in the output layer weights ($\mathbf{w}_{o}$): $64 \times 1 = 64$
- Number of parameters in the output layer biases ($\mathbf{b}_{o}$): $64 \times 1 = 1$

This results in a total of $(3200 + 64) + (4096 +64) + (2048 + 32) + (1024 +32) + (64 +1) = 10625 $ degrees of freedom (dofs) within our model.

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model=model, rankdir="LR", dpi=72, show_shapes=True, show_layer_activations=True
)

#### Model Compilation

Now we will compile our neural network model. Model compilation involves defining key components, such as the loss function, optimizer, learning rate, and metrics.

##### Loss Function

For our regression task, we use the Mean Absolute Percentage Error (MAPE) loss function. The *Mean Absolute Percentage Error loss is calculated as:

The Mean Absolute Percentage Error (MAPE) loss function can be represented in LaTeX as follows:

$$ MAPE = \frac{1}{n} \sum_{i=1}^{n} \left| \frac{y_i - \hat{y}_i}{y_i} \right| \times 100\% $$

where $ n $ is the number of data points, $ y_i $ represents the actual (observed) values, $ \hat{y}_i $ represents the predicted values.

In [ ]:
loss = losses.MeanAbsolutePercentageError()

##### Optimizer

We use the *Adam* optimizer, a popular choice for training neural networks. The [Adam optimization algorithm](https://doi.org/10.48550/arXiv.1412.6980) is a neural network-specific adaptation of the [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) method.

In [ ]:
optimizer = optimizers.Adam(learning_rate=1e-2, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

##### Metrics

Metrics are functions needed to measure the behavior of our model. There are many to choose from depending on the task of the model. For our case:

- **Root Mean Squared Error (RMSE)**: RMSE is a metric used to evaluate regression models. It measures the square root of the average of the squared differences between predicted and actual values. It quantifies the model's prediction error.

  $$ RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2} $$

  Where $n$ is the number of data points, $y_i$ represents the actual (observed) values and $\hat{y}_i$ represents the predicted values.

- **Mean Squared Error (MSE)**: MSE is another metric used to evaluate regression models. It is computed as the average of the squared differences between predicted and actual values.

  $$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$

  where $n$ is the number of data points, $y_i$ represents the actual (observed) values, and $\hat{y}_i$ represents the predicted values.


In [ ]:
metrics = [
    tf.keras.metrics.RootMeanSquaredError(),
    tf.keras.metrics.MeanSquaredError(),
]


##### Compilation

Finally, we compile the model by specifying the optimizer, loss function, and metrics.

In [ ]:
model.compile(
    optimizer,
    loss,
    metrics,
)

#### Training
In training, we define two key parameters:

In [ ]:
batch_size = 512
epochs = 500

* **Batch Size**: It specifies the number of training examples used in each iteration. A smaller batch size updates the model more frequently, while a larger one may improve training efficiency but requiring more volatile memory (RAM).

* **Epochs**: Each epoch represents one pass through the entire training dataset. It controls how many times the model iterates over the data, influencing convergence and potential overfitting.

Let's (finally) start the training process.

In [ ]:
with strategy.scope():
    # These initial lines of code are repeated because they need to be defined
    # inside the parallelization context to efficiently utilize the GPU/TPU.

    # Step 1: Building the Model
    model = build_model_2(features_train_try1.shape[1], "linear")

    # Step 2: Compiling the Model
    loss = losses.MeanAbsolutePercentageError()
    optimizer = optimizers.Adam(
        learning_rate=1e-2, beta_1=0.9, beta_2=0.999, epsilon=1e-08
    )
    metrics = [
        tf.keras.metrics.RootMeanSquaredError(),
        tf.keras.metrics.MeanSquaredError(),
    ]
    model.compile(
        optimizer,
        loss,
        metrics,
    )

    # Step 3: Training the Model
    history = model.fit(
        np.array(
            features_train_try1
        ),  # Convert the data into an array before feeding it
        np.array(labels_train_try1).astype("float"),
        batch_size,
        epochs,
        validation_data=(
            np.array(features_test_try1),
            np.array(labels_test_try1).astype("float"),
        ),  # Validation set
        verbose=1,  # 0 = silent, 1 = progress bar, 2 = one line per epoch
    )

#### Plot Training Progress

Now, we can plot the training progress.

In [ ]:
# @title Ausiliar plot function


def plot_training(
    ax,
    ax_twin,
    history,
    metric="root_mean_squared_error",
    metric_label="RMSE",
    halflife=25,
):
    """
    Plot training history with specified metric.

    Args:
        ax (Matplotlib Axis): The main plot axis.
        ax_twin (Matplotlib Axis): The twinned plot axis.
        history (Pandas DataFrame): Training history data.
        metric (str): The name of the metric to plot.
        metric_label (str): Label for the metric on the plot.
        halflife (int): Exponential moving average halflife for smoothing.

    Returns:
        None
    """
    ax.plot(history.index, history[metric], color="k", ls="-", alpha=0.25)
    a1 = ax.plot(
        history.index,
        history[metric].ewm(halflife=halflife).mean(),
        color="k",
        ls="-",
        label=metric_label + " (train)",
    )
    ax.plot(history.index, history["val_" + metric], color="k", ls="--", alpha=0.25)
    a2 = ax.plot(
        history.index,
        history["val_" + metric].ewm(halflife=halflife).mean(),
        color="k",
        ls="--",
        label=metric_label + " (test)",
    )
    ax_twin.plot(history.index, history["loss"], color="r", ls="-", alpha=0.25)
    l1 = ax_twin.plot(
        history.index,
        history["loss"].ewm(halflife=halflife).mean(),
        color="r",
        ls="-",
        label="Loss (train)",
    )
    ax_twin.plot(history.index, history["val_loss"], color="r", ls="--", alpha=0.25)
    l2 = ax_twin.plot(
        history.index,
        history["val_loss"].ewm(halflife=halflife).mean(),
        color="r",
        ls="--",
        label="Loss (test)",
    )
    ax.set_xlabel("Epochs [-]")
    ax.set_ylabel(metric_label + " [-]")
    ax_twin.set_ylabel(metric_label + "Loss [-]")
    ax_twin.legend(
        a1 + a2 + l1 + l2,
        [
            metric_label + " (train)",
            metric_label + " (test)",
            "Loss (train)",
            "Loss (test)",
        ],
        loc="upper right",
    )
    metric_data = np.concatenate(
        [
            history[metric].ewm(halflife=halflife).mean().values,
            history["val_" + metric].ewm(halflife=halflife).mean().values,
        ]
    )
    lb, ub = [np.percentile(metric_data, 0.5), np.percentile(metric_data, 99.5)]
    delta = ub - lb
    ax.set_ylim([lb - 0.05 * delta, ub + 0.05 * delta])
    loss_data = np.concatenate(
        [
            history["loss"].ewm(halflife=halflife).mean().values,
            history["val_loss"].ewm(halflife=halflife).mean().values,
        ]
    )
    lb, ub = [np.percentile(loss_data, 0.5), np.percentile(loss_data, 99.5)]
    delta = ub - lb
    ax_twin.set_ylim([lb - 0.05 * delta, ub + 0.05 * delta])


def get_rmsre(y_true, y_pred):
    """
    Get the Root Mean Square Relative Error.

    Args:
        y_true (array-like): True values.
        y_pred (array-like): Predicted values.

    Returns:
        RMSE (float): The Root Mean Square Relative Error.
    """
    return np.sqrt(np.mean(np.square((y_pred - y_true) / y_true)))


def get_mare(y_true, y_pred):
    """
    Get the Max Absolute Relative Error.

    Args:
        y_true (array-like): True values.
        y_pred (array-like): Predicted values.

    Returns:
        MARE (float): The Max Absolute Relative Error.
    """
    return np.max(np.abs(y_pred - y_true) / y_true)


def plot_regression(ax, axins, train_true, train_pred, test_true, test_pred):
    """
    Plot regression results and error distribution.

    Args:
        ax (Matplotlib Axis): The main plot axis.
        axins (Matplotlib Axis): The twinned plot axis.
        train_true (array-like): True values for the training set.
        train_pred (array-like): Predicted values for the training set.
        test_true (array-like): True values for the test set.
        test_pred (array-like): Predicted values for the test set.

    Returns:
        None
    """
    # compute error
    error_test_perc = np.abs(test_pred - test_true) / test_true * 100
    error_train_perc = np.abs(train_pred - train_true) / train_true * 100
    # compute error distibution
    p_test, bins_test = np.histogram(np.log(error_test_perc), bins=200, density=True)
    x_bins_test = (bins_test[1:] + bins_test[:-1]) / 2
    p_train, bins_train = np.histogram(np.log(error_train_perc), bins=200, density=True)
    x_bins_train = (bins_train[1:] + bins_train[:-1]) / 2
    # compute some more metrhics
    r2_test = r2_score(test_true, test_pred)
    r2_train = r2_score(train_true, train_pred)
    RMSRE_test = get_rmsre(test_true, test_pred) * 100.0
    RMSRE_train = get_rmsre(train_true, train_pred) * 100.0
    MARE_test = get_mare(test_true, test_pred) * 100.0
    MARE_train = get_mare(train_true, train_pred) * 100.0
    # train
    ax.scatter(train_true, train_pred, s=4, alpha=0.2, color="#126fbf", label="Train")
    axins.fill_between(
        np.exp(x_bins_train),
        p_train * 0.0,
        p_train,
        alpha=0.4,
        color="#126fbf",
        label="Train",
    )
    axins.plot(np.exp(x_bins_train), p_train, color="#126fbf", lw=1.2)
    # test
    ax.scatter(test_true, test_pred, s=4, alpha=0.2, color="#f3702b", label="Test")
    axins.fill_between(
        np.exp(x_bins_test),
        p_test * 0.0,
        p_test,
        alpha=0.4,
        color="#f3702b",
        label="Test",
    )
    axins.plot(np.exp(x_bins_test), p_test, color="#f3702b", lw=1.2)
    # additional metrics
    ax.annotate(
        f"Train:\n\t $R^2 = {r2_train:1.3f}"
        + f"$\n\t $RMSRE = {RMSRE_train:1.3f}$ % "
        + f"\n\t $MARE = {MARE_train:1.3f}$ % \n"
        + f"Test:\n\t $R^2 = {r2_test:1.3f}$"
        + f"\n\t $RMSRE = {RMSRE_test:1.3f}$ %"
        + f"\n\t $MARE = {MARE_test:1.3f}$ %",
        xy=(1.05, 0.5),
        xycoords=ax.transAxes,
        va="center",
    )
    ax.set_xlim([50e-2, 5e6])
    ax.set_ylim([50e-2, 5e6])
    ax.set_aspect("equal", adjustable="datalim")
    ax.axline([0, 0], [1, 1], color="k", ls="--", lw=0.75)
    ax.legend(loc="lower right")
    ax.set_xlabel(r"True $\mathrm{Re}$ [-]")
    ax.set_ylabel(r"Prediction $\mathrm{Re}$ [-]")
    axins.legend(loc="upper left")
    axins.set_xlabel("Relative error [%]")
    axins.set_ylabel("Probability density [-]")

In [ ]:
# Plot traing
training_history_try1 = pd.DataFrame(history.history)

In [ ]:
fig = plt.figure(figsize=(6, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    ax_twin = ax.twinx()
    plot_training(
        ax,
        ax_twin,
        training_history_try1,
        metric="root_mean_squared_error",
        metric_label="RMSE",
        halflife=20,
    )
    ax.set_yscale("log")
    ax_twin.set_yscale("log")
plt.show()

#### Model result analysis

Let's compare the model's predictions with the Reynolds number.

In [ ]:
# Predictions are generated using the model over the test set,
# which the model has never seen during training.
real_test_Re = np.array(labels_test_try1)
prediction_test_Re = model.predict(np.array(features_test_try1))
real_train_Re = np.array(labels_train_try1)
prediction_train_Re = model.predict(np.array(features_train_try1))

In [ ]:
fig = plt.figure(figsize=(5, 5))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    axins = ax.inset_axes([0.18, 0.65, 0.32, 0.32])
    plot_regression(
        ax, axins, real_train_Re, prediction_train_Re, real_test_Re, prediction_test_Re
    )
    ax.set_xscale("log")
    ax.set_yscale("log")
    axins.set_xscale("log")

### Exercise 1: Activation function

Let's explore some of the activation functions provided by`Tensorflow`:

In [ ]:
# List of activation function names
activation_functions = [
    "relu",
    "elu",
    "selu",
    "gelu",
    "swish",
    "mish",
    "linear",
    "exponential",
    "softplus",
    "tanh",
    "softsign",
    "sigmoid",
]

# Create TensorFlow constant input
x = tf.constant(np.linspace(-5, 5, 100), dtype=tf.float32)

# Create a grid of subplots
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(3, 4, hspace=0.4, wspace=0.3)
axs = gs.subplots()

# Plot activation functions
with plt.style.context("seaborn-v0_8-paper"):
    for i, activation_name in enumerate(activation_functions):
        row, col = i // 4, i % 4
        activation_func = getattr(tf.keras.activations, activation_name)
        y = activation_func(x)

        axs[row, col].plot(x, y, label=activation_name.capitalize())
        axs[row, col].set_xlabel("$x$ (-)")
        axs[row, col].set_ylabel("$f(x)$ (-)")
        axs[row, col].set_title(activation_name.capitalize())
        axs[row, col].grid()

plt.show()

For our regression model, determining the optimal activation function is essential. Let's perform a sensitivity analysis together by retraining the model with various activation functions available in TensorFlow. This exploration will help us identify the activation function that yields the best performance for our regression task

In [ ]:
ACTIVATION = "linear"  # <--- Place here the one with the best metrics

### Second Try: Increase inputs information

In this second experiment, we aim to explore the impact of increasing the dimensionality of the problem by incorporating additional information as inputs to the model, not limited to just the velocity profile.

Within our dataset, we possess valuable information about the physical properties of the fluid ($\mu \,\text{(Pas)}$, $\rho \,\mathrm{(kg/m^3)}$) and the geometric characteristics of the experimental setup ($L \,\text{(m)}$, $R \,\text{(m3)}$). These data are readily available for the model and can enhance its performance.

Furthermore, we can compute the partial derivative of the velocity profile $\dfrac{\partial u_y}{\partial x}$, which we will later discover is closely linked to the velocity stress tensor.

#### Architecture

Since the input shape changes drastically, as a rule of thumb, we increase the width of the model.

The sketch below represents the proposed neural network architecture.

<img style="display: block; margin: auto;" alt="NN sketch" src="https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/assets/img/NN_model2_try2.png">

#### Load data

In [ ]:
# Note: we are going to use all the data incluse the phisical properties (mu(Pas),	rho(kg/m3))
# and the gemetrical data (L(m),	R(m))
features_train_try2 = pd.read_csv(
    os.path.join("model_2", "train", "features.csv"), index_col=False
)
features_test_try2 = pd.read_csv(
    os.path.join("model_2", "test", "features.csv"), index_col=False
)
labels_train_try2 = pd.read_csv(
    os.path.join("model_2", "train", "labels.csv"), index_col=False
)
labels_test_try2 = pd.read_csv(
    os.path.join("model_2", "test", "labels.csv"), index_col=False
)

Let's increase futher more the input information computing the derivative

$$ \left[\nabla \mathbf{u} \right]_{ij} = \partial_i u_j =  \dfrac{\partial u_y}{\partial x} \approx \dfrac{u_y^n+1 - u_y^n}{\Delta x}$$

Note that for the Navier-Stoke for compressible fluids ($\rho = \rho_0 = \mathrm{const}(t)$):

$$ \rho_0\dfrac{\partial \mathbf{u}}{\partial t} + \rho_0(\mathbf{u} \cdot \nabla) \mathbf{u} = - \nabla p + \nabla\cdot\mathbf{\tau}   + \rho_0\mathbf{g}  $$

where the stress tensor $\mathbf{\tau}$
$$ \mathbf{\tau} =  \mu \left(\nabla\mathbf{u} + \left(\nabla\mathbf{u}\right)^\text{T} \right) $$

So what we are including as information is proportional to the $ij=ji$ component of the sress tensor.

In [ ]:
# Extract the velocity data and radii from the training and test datasets
velocity_train = features_train_try2.iloc[:, 4:].values
R_train = features_train_try2["R(m)"].values
velocity_test = features_test_try2.iloc[:, 4:].values
R_test = features_test_try2["R(m)"].values

# Determine the number of columns (velocity components) in the data
N = velocity_train.shape[1]

# Calculate the velocity gradient (dudx) for the training dataset
dudx_train = pd.DataFrame(
    data=(velocity_train[:, :-1] - velocity_train[:, 1:])
    / (R_train[..., np.newaxis].repeat(N - 1, axis=1) / N),
    columns=[f"dudx[{i:d}]" for i in range(N - 1)],
)

# Calculate the velocity gradient (dudx) for the test dataset
dudx_test = pd.DataFrame(
    data=(velocity_test[:, :-1] - velocity_test[:, 1:])
    / (R_test[..., np.newaxis].repeat(N - 1, axis=1) / N),
    columns=[f"dudx[{i:d}]" for i in range(N - 1)],
)

Let's glue the new data to the input features

In [ ]:
features_train_try2_full = pd.concat([features_train_try2, dudx_train], axis=1)
features_test_try2_full = pd.concat([features_test_try2, dudx_test], axis=1)

In [ ]:
features_train_try2_full.head()

#### Model building

In [ ]:
def build_model_2_wide(n_cols: int, out_activation: str) -> tf.keras.models.Sequential:
    """
    Build a the first possible architecture for our neural network model.

    Args:
        n_cols (int): Number of input features.
        out_activation (str): Output activation function.
    Returns:
        tf.keras.models.Sequential: A Keras Sequential model.
    """
    model = Sequential(
        [
            # Input layer with the specified input shape
            layers.InputLayer(input_shape=(n_cols,), name="input_layer"),
            # Add the first hidden layer with 64 perceptron and ReLU activation
            layers.Dense(128, activation="relu", name="hidden_layer_1"),
            # Add the second hidden layer with 64 perceptron and ReLU activation
            layers.Dense(128, activation="relu", name="hidden_layer_2"),
            # Add the third hidden layer with 64 perceptron and ReLU activation
            layers.Dense(64, activation="relu", name="hidden_layer_3"),
            # Add the fourth hidden layer with 64 perceptron and ReLU activation
            layers.Dense(64, activation="relu", name="hidden_layer_4"),
            # Add the output layer with a single perceptron (we expect a True/False answer) and sigmoid activation
            layers.Dense(1, activation=out_activation, name="output_layer"),
        ]
    )
    return model

Let's call the function to build our model.

In [ ]:
model_v2 = build_model_2_wide(features_train_try2_full.shape[1], ACTIVATION)

#### Model Summary and Structure Visualization

In [ ]:
model_v2.summary()

In [ ]:
tf.keras.utils.plot_model(
    model=model_v2, rankdir="LR", dpi=72, show_shapes=True, show_layer_activations=True
)

#### Model Compilation And Training

Now we will compile our neural network model. Model compilation involves defining key components, such as the loss function, optimizer, learning rate, and metrics. And then we train the new model.

In [ ]:
batch_size = 512
epochs = 500

In [ ]:
with strategy.scope():
    # These initla lines of code are repeated because it must be defined
    # inside the parallelization context for efficient GPU/TPU utilization.

    # Step 1: Building the Model
    model_v2 = build_model_2_wide(features_train_try2_full.shape[1], ACTIVATION)

    # Step 2: Compiling the Model
    loss = losses.MeanAbsolutePercentageError()
    optimizer = optimizers.Adam(
        learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08
    )
    metrics = [
        tf.keras.metrics.RootMeanSquaredError(),
        tf.keras.metrics.MeanSquaredError(),
    ]
    model_v2.compile(
        optimizer,
        loss,
        metrics,
    )

    # Step 3: Training the Model
    history = model_v2.fit(
        np.array(
            features_train_try2_full
        ),  # Convert the data into an array before feeding it
        np.array(labels_train_try2).astype("float"),
        batch_size,
        epochs,
        validation_data=(
            np.array(features_test_try2_full),
            np.array(labels_test_try2).astype("float"),
        ),  # Validation set
        verbose=1,  # 0 = silent, 1 = progress bar, 2 = one line per epoch
    )

#### Plot Training Progress

Now, we can plot the training progress.

In [ ]:
# Plot traing loop
training_history_try2 = pd.DataFrame(history.history)

In [ ]:
training_history_try2.head()

In [ ]:
fig = plt.figure(figsize=(6, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    ax_twin = ax.twinx()
    plot_training(
        ax,
        ax_twin,
        training_history_try2,
        metric="root_mean_squared_error",
        metric_label="RMSE",
        halflife=50,
    )
    ax.set_yscale("log")
    ax_twin.set_yscale("log")
plt.show()

#### Model result analysis

Let's compare the model's predictions with the Reynolds number.

In [ ]:
# Predictions are generated using the model over the test set,
# which the model has never seen during training.
real_test_Re = np.array(labels_test_try2)
prediction_test_Re = model_v2.predict(np.array(features_test_try2_full))
real_train_Re = np.array(labels_train_try2)
prediction_train_Re = model_v2.predict(np.array(features_train_try2_full))

In [ ]:
fig = plt.figure(figsize=(5, 5))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    axins = ax.inset_axes([0.18, 0.65, 0.32, 0.32])
    plot_regression(
        ax, axins, real_train_Re, prediction_train_Re, real_test_Re, prediction_test_Re
    )
    ax.set_xscale("log")
    ax.set_yscale("log")
    axins.set_xscale("log")

### Third Try: Normalization on the fly

#### Architecture

Since the input shape changes drastically, as a rule of thumb, we increase the width of the model.

The sketch below represents the proposed neural network architecture.

<img style="display: block; margin: auto;" alt="NN sketch" src="https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/assets/img/NN_model2_try3.png">

#### Load data

In [ ]:
features_train_try3 = pd.read_csv(
    os.path.join("model_2", "train", "features.csv"), index_col=False
).iloc[
    :, 4:
]  # Note: we drop the first 4 columns to study only the velocity profile
features_test_try3 = pd.read_csv(
    os.path.join("model_2", "test", "features.csv"), index_col=False
).iloc[
    :, 4:
]  # Note: we drop the first 4 columns to study only the velocity profile
labels_train_try3 = pd.read_csv(
    os.path.join("model_2", "train", "labels.csv"), index_col=False
)
labels_test_try3 = pd.read_csv(
    os.path.join("model_2", "test", "labels.csv"), index_col=False
)

#### Model building

##### Make custom *normalization* layer

It is important to normalize input data for neural network (NN) training for the following reasons:

* **Stability and Convergence**: Normalization helps in stabilizing the training process. When input features have significantly different scales, it can lead to slow convergence or the network getting stuck in local minima during training. By normalizing the data, the optimization algorithm can work more effectively and converge faster.

* **Gradient Descent**: During gradient descent, the learning rate plays a crucial role. If input features are not normalized, the learning rate might need to be adjusted manually to prevent divergence or slow convergence. Normalized data reduces the sensitivity to the learning rate, making it easier to find a good learning rate automatically.

* **Improved Generalization**: Normalization can lead to better generalization of the model.

* **Model Interpretability**: Normalized data can make it easier to interpret the importance of features in the model. Without normalization, the magnitude of feature weights in the neural network may not accurately reflect their importance.

Now, let's write a custom layer `VelocityNormalization`` that perform the normalization of the velocity profile, and at the same time, add the maximum velocity to the data.


In [ ]:
class VelocityNormalization(layers.Layer):
    def __init__(self, axis=1, name="velocity_normalization", **kwargs):
        super().__init__(name=name, **kwargs)
        self.axis = axis

    def call(self, batch):
        batch_max = tf.reduce_max(batch, axis=self.axis, keepdims=True)
        normalized_batch = batch / batch_max
        normalized_with_scale = tf.concat([normalized_batch, batch_max], axis=self.axis)
        return normalized_with_scale

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[self.axis] += 1  # One additional element for the scale vector
        return tuple(output_shape)

In [ ]:
# Convert the NumPy array to a TensorFlow constant
input_data_tf = tf.constant(features_train_try3.iloc[:4, :].values, dtype=tf.float32)

# Create a VelocityNormalization layer
batch_max_norm_layer = VelocityNormalization()

# Normalize the input data using the VelocityNormalization layer
normalized_data = batch_max_norm_layer(input_data_tf)

In [ ]:
# Create a plot of random data from the datasets
fig = plt.figure(figsize=(8, 3))
# Set the style of the plot to "seaborn-v0_8-paper"
with plt.style.context("seaborn-v0_8-paper"):
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # Plot the data and labels it with Re value
    for i in range(input_data_tf.shape[0]):
        ax1.plot(input_data_tf[i].numpy())
        ax2.plot(normalized_data[i].numpy()[:-1])

    # Adding labels and titles to the plot
    ax1.set_xlabel("i (-)")
    ax1.set_ylabel("$u$ (m/s)")
    ax2.set_xlabel("i (-)")
    ax2.set_ylabel("$u/u_{max}$ (-)")

In [ ]:
def build_model_2_norm(n_cols: int, out_activation: str) -> tf.keras.models.Sequential:
    """
    Build a the second possible architecture for our neural network model.

    Args:
        n_cols (int): Number of input features.
        out_activation (str): Output activation function.
    Returns:
        tf.keras.models.Sequential: A Keras Sequential model.
    """
    model = Sequential(
        [
            # Input layer with the specified input shape
            layers.InputLayer(input_shape=(n_cols,), name="input_layer"),
            # Normalization layer
            VelocityNormalization(name="normalization"),
            # Add the first hidden layer with 64 perceptron and ReLU activation
            layers.Dense(64, activation="relu", name="hidden_layer_1"),
            # Add the second hidden layer with 64 perceptron and ReLU activation
            layers.Dense(64, activation="relu", name="hidden_layer_2"),
            # Add the third hidden layer with 64 perceptron and ReLU activation
            layers.Dense(32, activation="relu", name="hidden_layer_3"),
            # Add the forth hidden layer with 64 perceptron and ReLU activation
            layers.Dense(32, activation="relu", name="hidden_layer_4"),
            # Add the output layer with a single perceptron (we expect a True/False answer) and sigmoid activation
            layers.Dense(1, activation=out_activation, name="output_layer"),
        ]
    )
    return model

In [ ]:
model_v3 = build_model_2_norm(features_train_try3.shape[1], ACTIVATION)

#### Model Summary and Structure Visualization



In [ ]:
model_v3.summary()

In [ ]:
tf.keras.utils.plot_model(
    model=model_v3, rankdir="LR", dpi=72, show_shapes=True, show_layer_activations=True
)

#### Model Compilation And Training

Now we will compile our neural network model. Model compilation involves defining key components, such as the loss function, optimizer, learning rate, and metrics. And then we train the new model.

In [ ]:
batch_size = 512
epochs = 500

In [ ]:
with strategy.scope():
    # These initla lines of code are repeated because it must be defined
    # inside the parallelization context for efficient GPU/TPU utilization.

    # Step 1: Building the Model
    model_v3 = build_model_2_norm(features_train_try3.shape[1], ACTIVATION)

    # Step 2: Compiling the Model
    loss = losses.MeanAbsolutePercentageError()
    optimizer = optimizers.Adam(
        learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08
    )
    metrics = [
        tf.keras.metrics.RootMeanSquaredError(),
        tf.keras.metrics.MeanSquaredError(),
    ]
    model_v3.compile(
        optimizer,
        loss,
        metrics,
    )

    # Step 3: Training the Model
    history = model_v3.fit(
        np.array(
            features_train_try3
        ),  # Convert the data into an array before feeding it
        np.array(labels_train_try3).astype("float"),
        batch_size,
        epochs,
        validation_data=(
            np.array(features_test_try3),
            np.array(labels_test_try3).astype("float"),
        ),  # Validation set
        verbose=1,  # 0 = silent, 1 = progress bar, 2 = one line per epoch
    )

#### Plot Training Progress

Now, we can plot the training progress.

In [ ]:
# Plot traing loop
training_history_try3 = pd.DataFrame(history.history)

In [ ]:
training_history_try3.head()

In [ ]:
fig = plt.figure(figsize=(6, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    ax_twin = ax.twinx()
    plot_training(
        ax,
        ax_twin,
        training_history_try3,
        metric="root_mean_squared_error",
        metric_label="RMSE",
        halflife=10,
    )
    # ax.set_yscale("log")
    ax_twin.set_yscale("log")
plt.show()

#### Model result analysis

Let's compare the model's predictions with the Reynolds number.

In [ ]:
# Predictions are generated using the model over the test set,
# which the model has never seen during training.
real_test_Re = np.array(labels_test_try3)
prediction_test_Re = model_v3.predict(np.array(features_test_try3))
real_train_Re = np.array(labels_train_try3)
prediction_train_Re = model_v3.predict(np.array(features_train_try3))

In [ ]:
fig = plt.figure(figsize=(5, 5))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    axins = ax.inset_axes([0.18, 0.65, 0.32, 0.32])
    plot_regression(
        ax, axins, real_train_Re, prediction_train_Re, real_test_Re, prediction_test_Re
    )
    ax.set_xscale("log")
    ax.set_yscale("log")
    axins.set_xscale("log")

### Exercise 2: Can NN predict Velocity profile

Knowing the physical properties of the fluid ($\mu \,\text{(Pas)}$, $\rho \,\mathrm{(kg/m^3)}$) and the geometric characteristics of the experimental setup ($L \,\text{(m)}$, $R \,\text{(m3)}$) and the Raynolds number ($\mathrm{Re} \,\mathrm{(-)}$), can we build a model that predict the velocity profile?

steps:
- Design a architecture
- Chose a loss function and metrics
- Compile the model
- Training
- Visualize the result